# Data Load, Cleaning

## Beginning by importing demographic features and existance of CKD

The following SQL queries filter by my inclusion and exclusion criteria 

#### Inclusion Critera
- Has recorded short read WGS data
- Has recorded EHR data
- Age 18-100
- Has a recorded BMI measurement
- Has a recorded systolic and diastolic blood pressure measurement
- Has a recorded lab value for BUN, blood glucose, sodium, and potassium
- Has answered the following surveys at least once: The Basics, Lifestyle, Overall Health, Healthcare Access and Utilization, Social Determinents of Health, Personal and Family Health History

#### Exclusion Criteria
- Has a cancer diagnosis 

In [ ]:
import pandas as pd
import os

# This query represents dataset "ML Features- CKD" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
dataset_94324895_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_occurrence.condition_source_value,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.vocabulary_id as source_vocabulary 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (1571486, 35209274, 35209275, 35209276, 35209277, 35209278, 35209279, 45548653, 725440, 725441, 725442)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id"""

dataset_94324895_condition_df = pd.read_gbq(
    dataset_94324895_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_94324895_condition_df.head(5)


# This query represents dataset "ML Features- CKD" for domain "person" and was generated for All of Us Controlled Tier Dataset v8
dataset_94324895_person_sql = """
    SELECT
        person.person_id,
        person.birth_datetime as date_of_birth,
        p_race_concept.concept_name as race,
        p_ethnicity_concept.concept_name as ethnicity,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                AND NOT EXISTS (      SELECT
                    'x'      
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                WHERE
                    d.person_id = p.person_id ) ) 
            AND cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_ehr_data = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                AND NOT EXISTS (      SELECT
                    'x'      
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                WHERE
                    d.person_id = p.person_id ) ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (3038553) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 0 
                        AND is_selectable = 1) 
                    AND is_standard = 0 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (3027018) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (903118, 903115) 
                    AND is_standard = 0 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (3013682) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (3004501) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (3019550) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (3023103) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (443392)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
            WHERE
                ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
            AND cb_search_person.person_id NOT IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
            WHERE
                sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
            AND cb_search_person.person_id NOT IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
            WHERE
                race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )"""

dataset_94324895_person_df = pd.read_gbq(
    dataset_94324895_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_94324895_person_df.head(5)

Merge the dataframes to more easily work with the data

In [ ]:
merged_df = (
   dataset_94324895_person_df.merge(dataset_94324895_condition_df, on="person_id", how="left")
)

In [ ]:
merged_df.shape

In [ ]:
merged_df.head(5)

Each patient has multiple rows. To retain only one for each patient, I am going to retain the most recent condition start in order to preserve the most recent observation for CKD. 

In [ ]:
latest_non_null = merged_df.loc[
    merged_df.dropna(subset = ["condition_start_datetime"])
    .groupby("person_id")["condition_start_datetime"]
    .idxmax()]

In [ ]:
missing_dates = merged_df[merged_df["condition_start_datetime"].isna()]

In [ ]:
ckd_features_df = pd.concat([latest_non_null, missing_dates])

In [ ]:
ckd_features_df["person_id"].unique()


#### Our total cohort size is 127,487 patients

Next, let's investigate how many patients have CKD based on ICD-10 code. 

In [ ]:
ckd_features_df["condition_start_datetime"].notna().sum()


Only 12,124 patients have chronic kidney disease. However, I am going to add more based on the GFR criteria. 

## Adding more CKD patients based on glomerular filtration rate 
- If a patient has a GFR < 60 for multiple lab tests, they will be added to my CKD cohort

In [ ]:
# This query represents dataset "ML Features- CKD" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v8
dataset_55367214_measurement_sql = """
    SELECT
        measurement.person_id,
        m_standard_concept.concept_name as standard_concept_name,
        measurement.measurement_datetime,
        measurement.value_as_number 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (1619025, 1619026, 3029859, 3030104, 3049187, 3053283, 36303797, 36306178, 40764999, 40771922, 46236952)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                measurement.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) measurement 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id"""

dataset_55367214_measurement_df = pd.read_gbq(
    dataset_55367214_measurement_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_55367214_measurement_df.head(5)

In [ ]:
low_gfr = dataset_55367214_measurement_df[
    (dataset_55367214_measurement_df['value_as_number'] < 60)
]


In [ ]:
low_gfr_counts = low_gfr.groupby('person_id').size()


In [ ]:
participants_with_low_gfr = low_gfr_counts[low_gfr_counts >= 2].index


In [ ]:
dataset_55367214_measurement_df['low_gfr'] = dataset_55367214_measurement_df['person_id'].apply(lambda x: 'yes' if x in participants_with_low_gfr else 'no')

In [ ]:
low_gfr_df = dataset_55367214_measurement_df[['person_id', 'low_gfr']]

In [ ]:
low_gfr_df = low_gfr_df.groupby('person_id').first().reset_index()

In [ ]:
low_gfr_df.head()

In [ ]:
low_gfr_df['person_id'].count()

In [ ]:
ckd_features_df = (
   ckd_features_df.merge(low_gfr_df, on="person_id", how="left")
)

In [ ]:
ckd_features_df.head()

# Feature cleaning

## Formatting the outcome variable (CKD) into 2 classes
0- No presence of CKD
1- presence of CKD 

In [ ]:
ckd_features_df["source_concept_name"].unique()

I am using ICD10 codes to identify those with CKD

In [ ]:
ckd_features_df['outcome'] = ckd_features_df['source_concept_name'].notna().astype(int).squeeze()


In [ ]:
ckd_features_df.head()

In [ ]:
ckd_features_df = ckd_features_df.drop(columns=['standard_concept_name', 'condition_end_datetime', 'condition_source_value', 'source_concept_name', 'source_vocabulary'])


In [ ]:
ckd_features_df.head()

In [ ]:
ckd_features_df['outcome'] = ckd_features_df.apply(
    lambda row: 1 if row['low_gfr'] == 'yes' else row['outcome'],
    axis=1
)

In [ ]:
ckd_features_df['outcome'].value_counts()

It is important to note that there is not a balance between our 2 classifications
This may necessary to address later in the ML process. 

## Cleaning sex variable

In [ ]:
ckd_features_df['sex_at_birth'].unique()

In [ ]:
ckd_features_df['sex'] = ckd_features_df['sex_at_birth']

In [ ]:
ckd_features_df = ckd_features_df.drop(columns = ['sex_at_birth'])

In [ ]:
ckd_features_df['sex'] = ckd_features_df['sex'].map({'Male': 0, 'Female': 1})

In [ ]:
ckd_features_df

## Calculating and binnning age variable

In [ ]:
from datetime import date

# Get today's date
today = date.today()

# Calculate age
ckd_features_df['Age'] = ckd_features_df['date_of_birth'].apply(
    lambda dob: today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
)

Making sure the minimum age is not below 18 and the maximum age is not above 100. 

In [ ]:
ckd_features_df['Age'].min()

In [ ]:
ckd_features_df['Age'].max()

### Binning the age variable

In [ ]:
ckd_features_df['Age'].hist()

In [ ]:
bins = [18, 25, 35, 45, 55, 65, 101]  
labels = ['Under 25', '25-34', '35-44', '45-54', '55-64', '65+']
# Create the age group column
ckd_features_df['Age_Group'] = pd.cut(ckd_features_df['Age'], bins=bins, labels=labels, right=False)

In [ ]:
ckd_features_df

In [ ]:
 ckd_features_df['Age_Group'].value_counts()

In [ ]:
import seaborn as sns
sns.histplot(ckd_features_df['Age_Group']);

We have very few participants under 25. The majority of the partipants are 65 and older. The rest of the age bin groups are fairly equally distributed. 

This will only be important for imputing missing values by age group. For the model, I will use the continuous age variable. 

In [ ]:
 ckd_features_df['condition_start_datetime'].count()

While I was going to calculate age at condition onset, only 12,000 patients have this data. Therefore, to preserve my large dataset, I am going to leave this feature out. 

In [ ]:
ckd_features_df_2 = ckd_features_df.drop(['date_of_birth', 'condition_start_datetime'], axis =1)

## Import BMI and Blood Pressure Data

In [ ]:
dataset_91352319_measurement_sql = """
    SELECT
        measurement.person_id,
        m_standard_concept.concept_name as standard_concept_name,
        measurement.measurement_datetime,
        measurement.value_as_number,
        measurement.unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (3004249, 3012888, 3038553, 37025979)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                measurement.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) measurement 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id"""

dataset_91352319_measurement_df = pd.read_gbq(
    dataset_91352319_measurement_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_91352319_measurement_df.head(5)

In [ ]:
labs_df = dataset_91352319_measurement_df.sort_values(by=['person_id', 'measurement_datetime'], ascending=[True, False])

# Group by 'patient_id' and 'standard_concept_name', and keep the most recent measurement
labs_df = labs_df.groupby(['person_id', 'standard_concept_name'], as_index=False).first()

labs_df.head(5)

In [ ]:
print(labs_df[['person_id', 'standard_concept_name', 'value_as_number']].isnull().sum())


Some patients do not have a recorded value for their labs. We are going to drop these patients. 

In [ ]:
labs_df = labs_df.dropna(subset=['value_as_number'])


In [ ]:
labs_df = labs_df.pivot(index='person_id', columns='standard_concept_name', values='value_as_number')

In [ ]:
labs_df.columns.name = None 
labs_df = labs_df.reset_index()

In [ ]:
labs_df

##### Merging the lab data with the rest of our eligible participants. 

In [ ]:
ckd_features_df_3 = (
   ckd_features_df_2.merge(labs_df, on="person_id", how="left")
)

In [ ]:
ckd_features_df_3

In [ ]:
ckd_features_df_3.isna().sum()

There are too many misisng glucose measurements, therefore these columns will be dropped. 

In [ ]:
ckd_features_df_3.drop(['Glucose mean value [Mass/volume] in Blood Estimated from glycated hemoglobin', 'Glucose mean value [Moles/volume] in Blood Estimated from glycated hemoglobin'], axis = 1)

## Clean the BMI feature

In [ ]:
ckd_features_df_3['Body mass index (BMI) [Ratio]'].max()

It is not plausible for someone to have a BMI this high. Therefore, I'm going to make that person's BMI NA and anyone else who has a BMI of over 70. This is the highest value I would typically see when viewing BMI charts online. Then I will impute the mean for those patients. 

In [ ]:
import numpy as np
ckd_features_df_3.loc[ckd_features_df_3['Body mass index (BMI) [Ratio]'] > 65, 'Body mass index (BMI) [Ratio]'] = np.nan


In [ ]:
ckd_features_df_3['Body mass index (BMI) [Ratio]'].max()

In [ ]:
ckd_features_df_3['Body mass index (BMI) [Ratio]'].hist()

In [ ]:
ckd_features_df_3['Body mass index (BMI) [Ratio]'].describe()

Having a BMI under 10 would be life threatening and I am assuming this is a data entry error. 

In [ ]:
ckd_features_df_3.loc[ckd_features_df_3['Body mass index (BMI) [Ratio]'] < 10, 'Body mass index (BMI) [Ratio]'] = np.nan


While it appears some patients were outliers in the BMI categoreis, we can now impute the mean. I am going to impute the mean based on age group. For example, a patient with an NA BMI who is 18 will end up with a different imputed value than a patient who is 65.  

In [ ]:
# Calculate the mean BMI for each age group
mean_bmi_by_age_group = ckd_features_df_3.groupby('Age_Group')['Body mass index (BMI) [Ratio]'].transform('mean')

# Impute the missing BMI values with the mean of the corresponding age group
ckd_features_df_3['Body mass index (BMI) [Ratio]'] = ckd_features_df_3['Body mass index (BMI) [Ratio]'].fillna(mean_bmi_by_age_group)

In [ ]:
ckd_features_df_3['Body mass index (BMI) [Ratio]'].describe()

In [ ]:
ckd_features_df_3.rename(columns={'Body mass index (BMI) [Ratio]': 'BMI'}, inplace=True)

In [ ]:
ckd_features_df_3['BMI'].hist()

## Cleaning systolic and diastolic blood pressure

In [ ]:
ckd_features_df_4 = ckd_features_df_3.rename(columns= {'Diastolic blood pressure': 'dbp', 'Systolic blood pressure': 'sbp'})

Visualizing diastolic blood pressure distribution

In [ ]:
ckd_features_df_4['dbp'].hist()

In [ ]:
ckd_features_df_4['dbp'].max()

In [ ]:
ckd_features_df_4['dbp'].min()

It would not be possible to have a diastolic blood pressure of 0. Anything below 30 would indicate a crisis. Additionally, anything over 180 would indicate a current life crisis. I will impute the mean by age group for any participants with readings like this. 

In [ ]:
ckd_features_df_4.loc[ckd_features_df_4['dbp'] < 30, 'dbp'] = np.nan


In [ ]:
ckd_features_df_4.loc[ckd_features_df_4['dbp'] >180, 'dbp'] = np.nan


In [ ]:
ckd_features_df_4['dbp'].isna().sum()

In [ ]:
ckd_features_df_4['dbp'].min()

In [ ]:
# Calculate the mean dbp for each age group
mean_dbp_by_age_group = ckd_features_df_4.groupby('Age_Group')['dbp'].transform('mean')

# Impute the missing dbp values with the mean of the corresponding age group
ckd_features_df_4['dbp'] = ckd_features_df_4['dbp'].fillna(mean_dbp_by_age_group)

In [ ]:
ckd_features_df_4['dbp'].hist()

Beginning work on systolic blood pressure

In [ ]:
ckd_features_df_4['sbp'].hist()

In [ ]:
print(ckd_features_df_4['sbp'].min(), ckd_features_df_4['sbp'].max())


Inserting NaN for implausible sbp and then imputing the mean by age group 

In [ ]:
ckd_features_df_4.loc[ckd_features_df_4['sbp'] < 40, 'sbp'] = np.nan
ckd_features_df_4.loc[ckd_features_df_4['sbp'] > 250, 'sbp'] = np.nan


In [ ]:
ckd_features_df_4['sbp'].isna().sum()

In [ ]:
# Calculate the mean sbp for each age group
mean_sbp_by_age_group = ckd_features_df_4.groupby('Age_Group')['sbp'].transform('mean')

# Impute the missing sbp values with the mean of the corresponding age group
ckd_features_df_4['sbp'] = ckd_features_df_4['sbp'].fillna(mean_sbp_by_age_group)

In [ ]:
ckd_features_df_4['sbp'].hist()

In [ ]:
print(ckd_features_df_4['sbp'].min(), ckd_features_df_4['sbp'].max())


In [ ]:
ckd_features_df_5 = ckd_features_df_4.drop(columns = ['Glucose mean value [Mass/volume] in Blood Estimated from glycated hemoglobin', 'Glucose mean value [Moles/volume] in Blood Estimated from glycated hemoglobin'])

## Importing lab data

In [ ]:
# This query represents dataset "ML Features- CKD" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v8
dataset_35505870_measurement_sql = """
    SELECT
        measurement.person_id,
        m_standard_concept.concept_name as standard_concept_name,
        measurement.measurement_datetime,
        measurement.value_as_number,
        m_unit.concept_name as unit_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (3004501, 3013682, 3016723, 3019550, 3023103, 3024561)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                measurement.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) measurement 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id"""

dataset_35505870_measurement_df = pd.read_gbq(
    dataset_35505870_measurement_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_35505870_measurement_df.head(5)

In [ ]:
labs2_df = dataset_35505870_measurement_df.sort_values(by=['person_id', 'measurement_datetime'], ascending=[True, False])

# Group by 'patient_id' and 'standard_concept_name', and keep the most recent measurement
labs2_df = labs2_df.groupby(['person_id', 'standard_concept_name'], as_index=False).first()

labs2_df.head(5)

In [ ]:
print(labs2_df[['person_id', 'standard_concept_name', 'value_as_number']].isnull().sum())


Drop the patients who don't have recorded lab values

In [ ]:
labs2_df = labs2_df.dropna(subset=['value_as_number'])

### Apply correct units for each of my lab measurements. Converting any that are different 

Some of our lab measurements have different units. Unless otherwise labeled, I'm assuming (according to the all of us data browser)
- albumin in grams/deciliter
- creatinine in milligrams/deciliter
- glucose in milligrams/deciliter
- potassium in millimole per liter
- sodium in millimole per liter


In [ ]:
unique_albumin_units = labs2_df[labs2_df['standard_concept_name'] == 'Albumin [Mass/volume] in Serum or Plasma']['unit_concept_name'].unique()
print(unique_albumin_units)

In [ ]:
albumin_mask = (labs2_df['standard_concept_name'] == 'Albumin [Mass/volume] in Serum or Plasma') & (labs2_df['unit_concept_name'] == 'milligram per milliliter')
labs2_df.loc[albumin_mask, 'value_as_number'] = labs2_df.loc[albumin_mask, 'value_as_number'] / 1000

In [ ]:
albumin_mask = (labs2_df['standard_concept_name'] == 'Albumin [Mass/volume] in Serum or Plasma') & (labs2_df['unit_concept_name'] == 'no value')
labs2_df.loc[albumin_mask, 'value_as_number'] = labs2_df.loc[albumin_mask, 'value_as_number'] / 10

In [ ]:
albumin_mask = (labs2_df['standard_concept_name'] == 'Albumin [Mass/volume] in Serum or Plasma') & (labs2_df['unit_concept_name'] == 'gram per liter')
labs2_df.loc[albumin_mask, 'value_as_number'] = labs2_df.loc[albumin_mask, 'value_as_number'] / 10

In [ ]:
unique_creatinine_units = labs2_df[labs2_df['standard_concept_name'] == 'Creatinine [Mass/volume] in Serum or Plasma']['unit_concept_name'].unique()
print(unique_creatinine_units)

I investigated all of the following values by unit label and they are the same unit. 
Unclear why some are labeled differently, but I am basing this decision on realistic lab measurements

In [ ]:
unique_glucose_units = labs2_df[labs2_df['standard_concept_name'] == 'Glucose [Mass/volume] in Serum or Plasma']['unit_concept_name'].unique()
print(unique_glucose_units)

In [ ]:
unique_potassium_units = labs2_df[labs2_df['standard_concept_name'] == 'Potassium [Moles/volume] in Serum or Plasma']['unit_concept_name'].unique()
print(unique_potassium_units)

In [ ]:
unique_sodium_units = labs2_df[labs2_df['standard_concept_name'] == 'Sodium [Moles/volume] in Serum or Plasma']['unit_concept_name'].unique()
print(unique_sodium_units)

In [ ]:
labs2_df = labs2_df.pivot(index='person_id', columns='standard_concept_name', values='value_as_number')

In [ ]:
labs2_df.columns.name = None 
labs2_df = labs2_df.reset_index()

Merging the new lab values with the rest of the features

In [ ]:
ckd_features_df_5 = (
   ckd_features_df_5.merge(labs2_df, on="person_id", how="left")
)

In [ ]:
ckd_features_df_5.count()

### Glucose 

In [ ]:
ckd_features_df_5.rename(columns={'Glucose [Mass/volume] in Serum or Plasma': 'Glucose'}, inplace=True)

In [ ]:
ckd_features_df_5['Glucose'].hist()

In [ ]:
ckd_features_df_5.loc[ckd_features_df_5['Glucose'] > 400, 'Glucose'] = np.nan
ckd_features_df_5.loc[ckd_features_df_5['Glucose'] < 40, 'Glucose'] = np.nan

In [ ]:
ckd_features_df_5['Glucose'].hist()

In [ ]:
ckd_features_df_5['Glucose'].isna().sum()

In [ ]:
# Calculate the mean glucose for each age group
mean_glucose_by_age_group = ckd_features_df_5.groupby('Age_Group')['Glucose'].transform('mean')

# Impute the missing glucose values with the mean of the corresponding age group
ckd_features_df_5['Glucose'] = ckd_features_df_5['Glucose'].fillna(mean_glucose_by_age_group)

In [ ]:
ckd_features_df_5['Glucose'].describe()

### Potassium

In [ ]:
ckd_features_df_5.rename(columns={'Potassium [Moles/volume] in Serum or Plasma': 'Potassium'}, inplace=True)

In [ ]:
ckd_features_df_5['Potassium'].hist()

In [ ]:
ckd_features_df_5.loc[ckd_features_df_5['Potassium'] > 7, 'Potassium'] = np.nan
ckd_features_df_5.loc[ckd_features_df_5['Potassium'] < 2.5, 'Potassium'] = np.nan

In [ ]:
ckd_features_df_5['Potassium'].hist()

In [ ]:
# Calculate the mean potassium for each age group
mean_potassium_by_age_group = ckd_features_df_5.groupby('Age_Group')['Potassium'].transform('mean')

# Impute the missing potassium values with the mean of the corresponding age group
ckd_features_df_5['Potassium'] = ckd_features_df_5['Potassium'].fillna(mean_potassium_by_age_group)

In [ ]:
ckd_features_df_5['Potassium'].describe()

In [ ]:
sns.histplot(ckd_features_df_5['Potassium']);

### Sodium

In [ ]:
ckd_features_df_5.rename(columns={'Sodium [Moles/volume] in Serum or Plasma': 'Sodium'}, inplace=True)

In [ ]:
ckd_features_df_5['Sodium'].hist()

In [ ]:
ckd_features_df_5['Sodium'].min()

In [ ]:
ckd_features_df_5['Sodium'].max()

In [ ]:
ckd_features_df_5.loc[ckd_features_df_5['Sodium'] > 170, 'Sodium'] = np.nan
ckd_features_df_5.loc[ckd_features_df_5['Sodium'] < 115, 'Sodium'] = np.nan

In [ ]:
# Calculate the mean sodium for each age group
mean_sodium_by_age_group = ckd_features_df_5.groupby('Age_Group')['Sodium'].transform('mean')

# Impute the missing sodium values with the mean of the corresponding age group
ckd_features_df_5['Sodium'] = ckd_features_df_5['Sodium'].fillna(mean_sodium_by_age_group)

In [ ]:
sns.histplot(ckd_features_df_5['Sodium']);

### Urea Nitrogen

In [ ]:
ckd_features_df_5.rename(columns={'Urea nitrogen [Mass/volume] in Serum or Plasma': 'Urea'}, inplace=True)

In [ ]:
ckd_features_df_5['Urea'].hist()

In [ ]:
ckd_features_df_5.loc[ckd_features_df_5['Urea'] > 60, 'Urea'] = np.nan
ckd_features_df_5.loc[ckd_features_df_5['Urea'] < 5, 'Urea'] = np.nan

In [ ]:
ckd_features_df_5['Urea'].isna().sum()

In [ ]:
# Calculate the mean BUN for each age group
mean_bun_by_age_group = ckd_features_df_5.groupby('Age_Group')['Urea'].transform('mean')

# Impute the missing BUN values with the mean of the corresponding age group
ckd_features_df_5['Urea'] = ckd_features_df_5['Urea'].fillna(mean_bun_by_age_group)

In [ ]:
sns.histplot(ckd_features_df_5['Urea']);

### Creatinine

In [ ]:
ckd_features_df_5.rename(columns={'Creatinine [Mass/volume] in Serum or Plasma': 'Creatinine'}, inplace=True)

In [ ]:
ckd_features_df_5['Creatinine'].hist()

In [ ]:
ckd_features_df_5['Creatinine'].min()

In [ ]:
ckd_features_df_5['Creatinine'].max()

In [ ]:
ckd_features_df_5.loc[ckd_features_df_5['Creatinine'] > 2.5, 'Creatinine'] = np.nan
ckd_features_df_5.loc[ckd_features_df_5['Creatinine'] < 0.1, 'Creatinine'] = np.nan

In [ ]:
# Calculate the mean creatinine for each age group
mean_creatinine_by_age_group = ckd_features_df_5.groupby('Age_Group')['Creatinine'].transform('mean')

# Impute the missing creatinine values with the mean of the corresponding age group
ckd_features_df_5['Creatinine'] = ckd_features_df_5['Creatinine'].fillna(mean_creatinine_by_age_group)

In [ ]:
sns.histplot(ckd_features_df_5['Creatinine']);

### Albumin

In [ ]:
ckd_features_df_5.rename(columns={'Albumin [Mass/volume] in Serum or Plasma': 'Albumin'}, inplace=True)

In [ ]:
ckd_features_df_5['Albumin'].hist()

In [ ]:
ckd_features_df_5['Albumin'].min()

In [ ]:
ckd_features_df_5['Albumin'].max()

In [ ]:
ckd_features_df_5.loc[ckd_features_df_5['Albumin'] > 7, 'Albumin'] = np.nan
ckd_features_df_5.loc[ckd_features_df_5['Albumin'] < 1, 'Albumin'] = np.nan

In [ ]:
# Calculate the mean Albumin for each age group
mean_albumin_by_age_group = ckd_features_df_5.groupby('Age_Group')['Albumin'].transform('mean')

# Impute the missing Albumin values with the mean of the corresponding age group
ckd_features_df_5['Albumin'] = ckd_features_df_5['Albumin'].fillna(mean_albumin_by_age_group)

In [ ]:
ckd_features_df_5['Albumin'].hist()

## Importing Lifestyle Data

In [ ]:
# This query represents dataset "ML Features- CKD" for domain "survey" and was generated for All of Us Controlled Tier Dataset v8
dataset_65281885_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (1585855)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  
        AND (
            answer.PERSON_ID IN (SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                    AND NOT EXISTS (      SELECT
                        'x'      
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                    WHERE
                        d.person_id = p.person_id ) ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_ehr_data = 1 ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_whole_genome_variant = 1 ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                    AND NOT EXISTS (      SELECT
                        'x'      
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                    WHERE
                        d.person_id = p.person_id ) ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3038553) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 0 
                            AND is_selectable = 1) 
                        AND is_standard = 0 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3027018) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (903118, 903115) 
                        AND is_standard = 0 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3013682) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3004501) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3019550) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3023103) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (443392)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                WHERE
                    ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                WHERE
                    sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                WHERE
                    race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
        )"""

dataset_65281885_survey_df = pd.read_gbq(
    dataset_65281885_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_65281885_survey_df.head(5)

In [ ]:
dataset_65281885_survey_df['question'].unique()

In [ ]:
lifestyle_values = dataset_65281885_survey_df.drop(columns=['survey', 'question_concept_id', 'answer_concept_id', 'survey_version_concept_id', 'survey_version_name'])

In [ ]:
questions_to_keep = ['Smoking: 100 Cigs Lifetime', 'Alcohol: Alcohol Participant', 'Alcohol: Drink Frequency Past Year'] 
lifestyle_values = lifestyle_values[lifestyle_values['question'].isin(questions_to_keep)]


In [ ]:
lifestyle_values.head()

Keep most recent recorded survey answer

In [ ]:
lifestyle_values = lifestyle_values.sort_values(by=['person_id', 'survey_datetime'], ascending=[True, False])

# Group by 'patient_id' and 'standard_concept_name', and keep the most recent measurement
lifestyle_values = lifestyle_values.groupby(['person_id', 'question'], as_index=False).first()

lifestyle_values.head(5)

In [ ]:
 lifestyle_values['question'].value_counts()

All of our participants should have some answer for the alcohol and smoking questions. 
- Imporant note: smoking 100 cigs lifetime was the only smoking related question that had enough responses to include in my analysis

In [ ]:
lifestyle_values = lifestyle_values.drop(columns = ['survey_datetime']).pivot(index='person_id', columns='question', values='answer').reset_index()


In [ ]:
lifestyle_values.columns.name = None 

In [ ]:
lifestyle_values.head(10)

In [ ]:
lifestyle_values.loc[lifestyle_values['Alcohol: Alcohol Participant'] == 'Alcohol Participant: No', 'Alcohol: Drink Frequency Past Year'] = '0'


In [ ]:
lifestyle_values['Alcohol: Drink Frequency Past Year'].value_counts()

In [ ]:
def scale_score_drink_past_year(answer):
    if answer == 0 or answer == "0":
        return 0
    mapping = {
        "Drink Frequency Past Year: Never": 0,
        "Drink Frequency Past Year: Monthly Or Less": 1,
        "Drink Frequency Past Year: 2 to 4 Per Month": 2,
        "Drink Frequency Past Year: 2 to 3 Per Week": 3,
        "Drink Frequency Past Year: 4 or More Per Week": 4
    }
    return mapping.get(answer, None)

In [ ]:
lifestyle_values['alcohol_score'] = lifestyle_values['Alcohol: Drink Frequency Past Year'].apply(scale_score_drink_past_year)

In [ ]:
lifestyle_values = lifestyle_values.drop(columns=['Alcohol: Drink Frequency Past Year', 'Alcohol: Alcohol Participant'])

####  Merge with the rest of the feature set

In [ ]:
ckd_features_df_6 = (
   ckd_features_df_5.merge(lifestyle_values, on="person_id", how="left")
)

In [ ]:
ckd_features_df_6.head(10)

### Smoking Status 

In [ ]:
ckd_features_df_6.rename(columns={'Smoking: 100 Cigs Lifetime': 'smoking_status'}, inplace=True)

In [ ]:
ckd_features_df_6['smoking_status'].value_counts()

While this may oversimplify the smoking status variable, I will code "Prefer not to Answer", "Skip", and "Don't Know" participants as nonsmokers. 

This should be kept in mind as a potential limitation later. 

In [ ]:
ckd_features_df_6['smoking_status'] = ckd_features_df_6['smoking_status'].map({
    '100 Cigs Lifetime: Yes': 1,
    '100 Cigs Lifetime: No': 0,
    'PMI: Dont Know': 0,
    'PMI: Skip': 0,
    'PMI: Prefer Not To Answer': 0
})

In [ ]:
ckd_features_df_6['smoking_status'].isna().sum()

Imputing mode smoking status by age group for participants that are missing that survey answer

In [ ]:
def mode_imputer(series):
    return series.mode().iloc[0] if not series.mode().empty else np.nan

smoking_status_mode_by_age_group = ckd_features_df_6.groupby('Age_Group')['smoking_status'].apply(mode_imputer)


ckd_features_df_6['smoking_status'] = ckd_features_df_6.apply(
    lambda row: smoking_status_mode_by_age_group[row['Age_Group']] if pd.isna(row['smoking_status']) else row['smoking_status'],
    axis=1
)

In [ ]:
ckd_features_df_6['smoking_status'].isna().sum()

### Alcohol Use

Imputing mode alcohol score by age group for participants that are missing that survey answer

In [ ]:
alcohol_score_mode_by_age_group = ckd_features_df_6.groupby('Age_Group')['alcohol_score'].apply(mode_imputer)


ckd_features_df_6['alcohol_score'] = ckd_features_df_6.apply(
    lambda row: alcohol_score_mode_by_age_group[row['Age_Group']] if pd.isna(row['alcohol_score']) else row['alcohol_score'],
    axis=1
)

In [ ]:
sns.histplot(ckd_features_df_6['alcohol_score']);

## Overall Health Survey Features

In [ ]:

# This query represents dataset "ML Features- CKD" for domain "survey" and was generated for All of Us Controlled Tier Dataset v8
dataset_89356137_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.question,
        answer.answer  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (1585710)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  
        AND (
            answer.PERSON_ID IN (SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                    AND NOT EXISTS (      SELECT
                        'x'      
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                    WHERE
                        d.person_id = p.person_id ) ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_ehr_data = 1 ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_whole_genome_variant = 1 ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                    AND NOT EXISTS (      SELECT
                        'x'      
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                    WHERE
                        d.person_id = p.person_id ) ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3038553) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 0 
                            AND is_selectable = 1) 
                        AND is_standard = 0 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3027018) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (903118, 903115) 
                        AND is_standard = 0 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3013682) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3004501) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3019550) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3023103) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (443392)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                WHERE
                    ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                WHERE
                    sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                WHERE
                    race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
        )"""

dataset_89356137_survey_df = pd.read_gbq(
    dataset_89356137_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_89356137_survey_df.head(5)

In [ ]:
dataset_89356137_survey_df['question'].unique()

In [ ]:
questions_to_keep = ['Overall Health: General Mental Health', 'Overall Health: General Physical Health'] 
overall_health_values = dataset_89356137_survey_df[dataset_89356137_survey_df['question'].isin(questions_to_keep)]


In [ ]:
overall_health_values = overall_health_values.sort_values(by=['person_id', 'survey_datetime'], ascending=[True, False])

# Group by 'patient_id' and 'standard_concept_name', and keep the most recent measurement
overall_health_values = overall_health_values.groupby(['person_id', 'question'], as_index=False).first()

overall_health_values.head(5)

In [ ]:
overall_health_values['question'].value_counts()

In [ ]:
overall_health_values = overall_health_values.drop(columns = ['survey_datetime']).pivot(index='person_id', columns='question', values='answer').reset_index()

In [ ]:
overall_health_values.columns.name = None 

In [ ]:
overall_health_values.head()

In [ ]:
overall_health_values['mental_health'] = overall_health_values['Overall Health: General Mental Health']
overall_health_values['physical_health'] = overall_health_values['Overall Health: General Physical Health']

Both of these variables are ordinal and I will map them accordingly

In [ ]:
overall_health_values['mental_health'] = overall_health_values['mental_health'].map({
    'General Mental Health: Poor': 0,
    'General Mental Health: Fair': 1,
    'General Mental Health: Good': 2,
    'General Mental Health: Very Good': 3,
    'General Mental Health: Excllent': 4,
    'General Mental Health: Excellent': 4
})

In [ ]:
overall_health_values['physical_health'] = overall_health_values['physical_health'].map({
    'General Physical Health: Poor': 0,
    'General Physical Health: Fair': 1,
    'General Physical Health: Good': 2,
    'General Physical Health: Very Good': 3,
    'General Physical Health: Excellent': 4
})

In [ ]:
overall_health_values.head(20)

In [ ]:
overall_health_values = overall_health_values.drop(columns=['Overall Health: General Mental Health', 'Overall Health: General Physical Health'])

In [ ]:
ckd_features_df_7 = (
   ckd_features_df_6.merge(overall_health_values, on="person_id", how="left")
)

### Imputing mental health values

Imputing mode of mental health rating by age group. 

In [ ]:
ckd_features_df_7['mental_health'].isna().sum()

In [ ]:
def mode_imputer(series):
    return series.mode().iloc[0] if not series.mode().empty else np.nan

mental_health_mode_by_age_group = ckd_features_df_7.groupby('Age_Group')['mental_health'].apply(mode_imputer)


ckd_features_df_7['mental_health'] = ckd_features_df_7.apply(
    lambda row: mental_health_mode_by_age_group[row['Age_Group']] if pd.isna(row['mental_health']) else row['mental_health'],
    axis=1
)

In [ ]:
sns.histplot(ckd_features_df_7['mental_health']);

### Imputing physical health values

In [ ]:
ckd_features_df_7['physical_health'].isna().sum()

In [ ]:
physical_health_mode_by_age_group = ckd_features_df_7.groupby('Age_Group')['physical_health'].apply(mode_imputer)


ckd_features_df_7['physical_health'] = ckd_features_df_7.apply(
    lambda row: physical_health_mode_by_age_group[row['Age_Group']] if pd.isna(row['physical_health']) else row['physical_health'],
    axis=1
)

In [ ]:
sns.histplot(ckd_features_df_7['physical_health']);

## Healthcare Access Feature 

In [ ]:

# This query represents dataset "ML Features- CKD" for domain "survey" and was generated for All of Us Controlled Tier Dataset v8
dataset_31480590_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.question,
        answer.answer  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (43528895)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  
        AND (
            answer.PERSON_ID IN (SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                    AND NOT EXISTS (      SELECT
                        'x'      
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                    WHERE
                        d.person_id = p.person_id ) ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_ehr_data = 1 ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_whole_genome_variant = 1 ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                    AND NOT EXISTS (      SELECT
                        'x'      
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                    WHERE
                        d.person_id = p.person_id ) ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3038553) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 0 
                            AND is_selectable = 1) 
                        AND is_standard = 0 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3027018) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (903118, 903115) 
                        AND is_standard = 0 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3013682) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3004501) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3019550) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN (3023103) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    criteria.person_id 
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (443392)       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                WHERE
                    ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                WHERE
                    sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                AND cb_search_person.person_id NOT IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                WHERE
                    race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
        )"""

dataset_31480590_survey_df = pd.read_gbq(
    dataset_31480590_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_31480590_survey_df.head(5)

In [ ]:
dataset_31480590_survey_df['question'].unique()

In [ ]:
questions_to_keep = ["Can't Afford Care: Healthcare Provider"] 
afford_care_values = dataset_31480590_survey_df[dataset_31480590_survey_df['question'].isin(questions_to_keep)]

In [ ]:
afford_care_values

In [ ]:
afford_care_values = afford_care_values.sort_values(by=['person_id', 'survey_datetime'], ascending=[True, False])

# Group by 'patient_id' and 'question', and keep the most recent survey answer
afford_care_values = afford_care_values.groupby(['person_id', 'question'], as_index=False).first()

afford_care_values.head(5)

In [ ]:
afford_care_values = afford_care_values.drop(columns = ['survey_datetime']).pivot(index='person_id', columns='question', values='answer').reset_index()

In [ ]:
afford_care_values.columns.name = None 

In [ ]:
afford_care_values.rename(columns={"Can't Afford Care: Healthcare Provider": 'afford_provider'}, inplace=True)
afford_care_values.head(10)

In [ ]:
afford_care_values['afford_provider'].value_counts()

If a patient doesn't know if they can afford a healthcare provider, specifically a primary care provider, they likely can't or don't have an established PCP. Therefore, I will classify these patients as not able to afford a healthcare provider. 

In [ ]:
afford_care_values['afford_provider'] = afford_care_values['afford_provider'].map({
    "Can't Afford Healthcare Provider: Yes": 0,
    "Can't Afford Healthcare Provider: No": 1,
    'PMI: Skip': np.nan,
    'PMI: Dont Know': 0
})

In [ ]:
ckd_features_df_8 = (
   ckd_features_df_7.merge(afford_care_values, on="person_id", how="left")
)

In [ ]:
ckd_features_df_8.head()

In [ ]:
afford_care_mode_by_age_group = ckd_features_df_8.groupby('Age_Group')['afford_provider'].apply(mode_imputer)


ckd_features_df_8['afford_provider'] = ckd_features_df_8.apply(
    lambda row: afford_care_mode_by_age_group[row['Age_Group']] if pd.isna(row['afford_provider']) else row['afford_provider'],
    axis=1
)

In [ ]:
sns.histplot(ckd_features_df_8['afford_provider']);

There are significatly more patients that can afford a provider. This may make it difficult for the model to pick up on a any predictive power not affording a provider could have. This will be important to keep in mind when investigating feature importance.  

## Diabetes Mellitus

In [ ]:

# This query represents dataset "ML Features- CKD" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
dataset_30560220_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_standard_concept.concept_name as standard_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (201820, 201826)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id"""

dataset_30560220_condition_df = pd.read_gbq(
    dataset_30560220_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_30560220_condition_df.head(5)

In [ ]:
dataset_30560220_condition_df['standard_concept_name'].unique()

In [ ]:
diabetes_df = (
    dataset_30560220_condition_df
    .assign(diabetes=dataset_30560220_condition_df['standard_concept_name'].notna().astype(int))
    .groupby('person_id', as_index=False)['diabetes']
    .max()
)

In [ ]:
ckd_features_df_9 = (
   ckd_features_df_8.merge(diabetes_df, on="person_id", how="left")
)

In [ ]:
ckd_features_df_9['diabetes'].isna().sum()

In [ ]:
ckd_features_df_9['diabetes'] = ckd_features_df_9['diabetes'].fillna(0)

In [ ]:
ckd_features_df_9['diabetes'].value_counts()

## Hypertension

In [ ]:
# This query represents dataset "ML Features- CKD" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
dataset_01165059_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_standard_concept.concept_name as standard_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (316866, 320128)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id"""

dataset_01165059_condition_df = pd.read_gbq(
    dataset_01165059_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_01165059_condition_df.head(5)

In [ ]:
hypertension_df = (
    dataset_01165059_condition_df
    .assign(hypertension=dataset_01165059_condition_df['standard_concept_name'].notna().astype(int))
    .groupby('person_id', as_index=False)['hypertension']
    .max()
)

In [ ]:
ckd_features_df_10 = (
   ckd_features_df_9.merge(hypertension_df, on="person_id", how="left")
)

In [ ]:
ckd_features_df_10['hypertension'].isna().sum()

In [ ]:
ckd_features_df_10['hypertension'] = ckd_features_df_10['hypertension'].fillna(0)

In [ ]:
ckd_features_df_10['hypertension'].value_counts()

## Add more lab values 

In [ ]:
# This query represents dataset "ML Features- CKD" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v8
dataset_50879404_measurement_sql = """
    SELECT
        measurement.person_id,
        m_standard_concept.concept_name as standard_concept_name,
        measurement.measurement_datetime,
        measurement.value_as_number,
        m_unit.concept_name as unit_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (3000905, 3000963, 3020416)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                measurement.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) measurement 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id"""

dataset_50879404_measurement_df = pd.read_gbq(
    dataset_50879404_measurement_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_50879404_measurement_df.head(5)

In [ ]:
more_labs_df = dataset_50879404_measurement_df.sort_values(by=['person_id', 'measurement_datetime'], ascending=[True, False])

# Group by 'patient_id' and 'standard_concept_name', and keep the most recent measurement
more_labs_df = more_labs_df.groupby(['person_id', 'standard_concept_name'], as_index=False).first()

more_labs_df.head(5)

In [ ]:
print(more_labs_df[['person_id', 'standard_concept_name', 'value_as_number']].isnull().sum())


In [ ]:
more_labs_df = more_labs_df.dropna(subset=['value_as_number'])

#### Ensuring units are consistent 

In [ ]:
unique_rbc_units = more_labs_df[more_labs_df['standard_concept_name'] == 'Erythrocytes [#/volume] in Blood by Automated count']['unit_concept_name'].unique()
print(unique_rbc_units)

In [ ]:
unique_hem_units = more_labs_df[more_labs_df['standard_concept_name'] == 'Hemoglobin [Mass/volume] in Blood']['unit_concept_name'].unique()
print(unique_hem_units)

In [ ]:
hemoglobin_mask = (more_labs_df['standard_concept_name'] == 'Hemoglobin [Mass/volume] in Blood') & (more_labs_df['unit_concept_name'] == 'gram per liter')
more_labs_df.loc[hemoglobin_mask, 'value_as_number'] = more_labs_df.loc[hemoglobin_mask, 'value_as_number'] / 10

In [ ]:
unique_wbc_units = more_labs_df[more_labs_df['standard_concept_name'] == 'Leukocytes [#/volume] in Blood by Automated count']['unit_concept_name'].unique()
print(unique_wbc_units)

In [ ]:
wbc_mask = (more_labs_df['standard_concept_name'] == 'Leukocytes [#/volume] in Blood by Automated count') & (more_labs_df['unit_concept_name'] == 'per cubic millimeter')
more_labs_df.loc[wbc_mask, 'value_as_number'] = more_labs_df.loc[wbc_mask, 'value_as_number'] / 1000

In [ ]:
more_labs_df = more_labs_df.pivot(index='person_id', columns='standard_concept_name', values='value_as_number')

In [ ]:
more_labs_df.columns.name = None 

In [ ]:
more_labs_df = more_labs_df.reset_index()


In [ ]:
more_labs_df.head()

In [ ]:
ckd_features_df_11 = (
   ckd_features_df_10.merge(more_labs_df, on="person_id", how="left")
)

In [ ]:
ckd_features_df_11.isna().sum()

### Hemoglobin

In [ ]:
ckd_features_df_11.rename(columns={'Hemoglobin [Mass/volume] in Blood': 'Hemoglobin'}, inplace=True)

In [ ]:
ckd_features_df_11['Hemoglobin'].hist()

In [ ]:
ckd_features_df_11['Hemoglobin'].min()

In [ ]:
ckd_features_df_11['Hemoglobin'].max()

In [ ]:
ckd_features_df_11.loc[ckd_features_df_11['Hemoglobin'] > 20, 'Hemoglobin'] = np.nan
ckd_features_df_11.loc[ckd_features_df_11['Hemoglobin'] < 8, 'Hemoglobin'] = np.nan

In [ ]:
# Calculate the mean Hemoglobin for each age group
mean_hem_by_age_group = ckd_features_df_11.groupby('Age_Group')['Hemoglobin'].transform('mean')
# Impute the missing Hemoglobin values with the mean of the corresponding age group
ckd_features_df_11['Hemoglobin'] = ckd_features_df_11['Hemoglobin'].fillna(mean_hem_by_age_group)


In [ ]:
ckd_features_df_11['Hemoglobin'].hist()

### Leukocytes 

In [ ]:
ckd_features_df_11.rename(columns={'Leukocytes [#/volume] in Blood by Automated count': 'Leukocytes'}, inplace=True)

In [ ]:
ckd_features_df_11['Leukocytes'].hist()

In [ ]:
ckd_features_df_11['Leukocytes'].min()

In [ ]:
ckd_features_df_11['Leukocytes'].max()

In [ ]:
ckd_features_df_11.loc[ckd_features_df_11['Leukocytes'] > 22, 'Leukocytes'] = np.nan
ckd_features_df_11.loc[ckd_features_df_11['Leukocytes'] < 1, 'Leukocytes'] = np.nan

In [ ]:
# Calculate the mean Leukocytes for each age group
mean_wbc_by_age_group = ckd_features_df_11.groupby('Age_Group')['Leukocytes'].transform('mean')
# Impute the missing Leukocytes values with the mean of the corresponding age group
ckd_features_df_11['Leukocytes'] = ckd_features_df_11['Leukocytes'].fillna(mean_wbc_by_age_group)

In [ ]:
ckd_features_df_11['Leukocytes'].hist()

### Erythrocytes 

In [ ]:
ckd_features_df_11.rename(columns={'Erythrocytes [#/volume] in Blood by Automated count': 'Erythrocytes'}, inplace=True)

In [ ]:
ckd_features_df_11['Erythrocytes'].hist()

In [ ]:
ckd_features_df_11['Erythrocytes'].min()

In [ ]:
ckd_features_df_11['Erythrocytes'].max()

In [ ]:
ckd_features_df_11.loc[ckd_features_df_11['Erythrocytes'] > 8, 'Erythrocytes'] = np.nan
ckd_features_df_11.loc[ckd_features_df_11['Erythrocytes'] < 1, 'Erythrocytes'] = np.nan

In [ ]:
# Calculate the mean Erythrocytes for each age group
mean_rbc_by_age_group = ckd_features_df_11.groupby('Age_Group')['Erythrocytes'].transform('mean')
# Impute the missing Erythrocytes values with the mean of the corresponding age group
ckd_features_df_11['Erythrocytes'] = ckd_features_df_11['Erythrocytes'].fillna(mean_rbc_by_age_group)

In [ ]:
ckd_features_df_11['Erythrocytes'].hist()

### Anemia

In [ ]:
# This query represents dataset "ML Features- CKD" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
dataset_04478998_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_standard_concept.concept_name as standard_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (439777)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id"""

dataset_04478998_condition_df = pd.read_gbq(
    dataset_04478998_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_04478998_condition_df.head(5)

In [ ]:
anemia_df = (
    dataset_04478998_condition_df
    .assign(anemia=dataset_04478998_condition_df['standard_concept_name'].notna().astype(int))
    .groupby('person_id', as_index=False)['anemia']
    .max()
)

In [ ]:
ckd_features_df_12 = (
   ckd_features_df_11.merge(anemia_df, on="person_id", how="left")
)

In [ ]:
ckd_features_df_12['anemia'].isna().sum()

In [ ]:
ckd_features_df_12['anemia'] = ckd_features_df_12['anemia'].fillna(0)

In [ ]:
ckd_features_df_12['anemia'].value_counts()

## Calcium, chloride, and alkaline phosphate lab values

In [ ]:
# This query represents dataset "ML Features- CKD" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v8
dataset_35753952_measurement_sql = """
    SELECT
        measurement.person_id,
        m_standard_concept.concept_name as standard_concept_name,
        measurement.measurement_datetime,
        measurement.value_as_number,
        m_unit.concept_name as unit_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (3006906, 3014576, 3035995)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                measurement.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) measurement 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id"""

dataset_35753952_measurement_df = pd.read_gbq(
    dataset_35753952_measurement_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_35753952_measurement_df.head(5)

In [ ]:
labs_3 = dataset_35753952_measurement_df.sort_values(by=['person_id', 'measurement_datetime'], ascending=[True, False])

# Group by 'patient_id' and 'standard_concept_name', and keep the most recent measurement
labs_3 = labs_3.groupby(['person_id', 'standard_concept_name'], as_index=False).first()

labs_3.head(5)

In [ ]:
print(labs_3[['person_id', 'standard_concept_name', 'value_as_number']].isnull().sum())

In [ ]:
labs_3 = labs_3.dropna(subset=['value_as_number'])

In [ ]:
unique_calcium_units = labs_3[labs_3['standard_concept_name'] == 'Calcium [Mass/volume] in Serum or Plasma']['unit_concept_name'].unique()
print(unique_calcium_units)

In [ ]:
calcium_mask = (labs_3['standard_concept_name'] == 'Calcium [Mass/volume] in Serum or Plasma') & (labs_3['unit_concept_name'] ==  'milliliter per minute')
labs_3.loc[calcium_mask, 'value_as_number'] = labs_3.loc[calcium_mask, 'value_as_number'] / 10

In [ ]:
unique_chloride_units = labs_3[labs_3['standard_concept_name'] == 'Chloride [Moles/volume] in Serum or Plasma']['unit_concept_name'].unique()
print(unique_chloride_units)

In [ ]:
unique_alk_units = labs_3[labs_3['standard_concept_name'] == 'Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma']['unit_concept_name'].unique()
print(unique_alk_units)

In [ ]:
alk_mask = (labs_3['standard_concept_name'] == 'Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma') & (labs_3['unit_concept_name'] == 'microgram per liter')
labs_3.loc[alk_mask, 'value_as_number'] = labs_3.loc[alk_mask, 'value_as_number'] * 10

In [ ]:
labs_3 = labs_3.pivot(index='person_id', columns='standard_concept_name', values='value_as_number')

In [ ]:
labs_3.columns.name = None 

In [ ]:
labs_3 = labs_3.reset_index()

In [ ]:
labs_3.head()

In [ ]:
ckd_features_df_13 = (
   ckd_features_df_12.merge(labs_3, on="person_id", how="left")
)

ckd_features_df_13.isna().sum()


## Calcium 

In [ ]:
ckd_features_df_13.rename(columns={'Calcium [Mass/volume] in Serum or Plasma': 'Calcium'}, inplace=True)

In [ ]:
ckd_features_df_13['Calcium'].hist()

In [ ]:
ckd_features_df_13['Calcium'].min()

In [ ]:
ckd_features_df_13['Calcium'].max()

In [ ]:
ckd_features_df_13.loc[ckd_features_df_13['Calcium'] > 14, 'Calcium'] = np.nan
ckd_features_df_13.loc[ckd_features_df_13['Calcium'] < 4, 'Calcium'] = np.nan

In [ ]:
# Calculate the mean calcium for each age group
mean_calcium_by_age_group = ckd_features_df_13.groupby('Age_Group')['Calcium'].transform('mean')
# Impute the missing calcium values with the mean of the corresponding age group
ckd_features_df_13['Calcium'] = ckd_features_df_13['Calcium'].fillna(mean_calcium_by_age_group)

In [ ]:
ckd_features_df_11['Hemoglobin'].hist()

## Chloride

In [ ]:
ckd_features_df_13.rename(columns={'Chloride [Moles/volume] in Serum or Plasma': 'Chloride'}, inplace=True)

In [ ]:
ckd_features_df_13['Chloride'].hist()

In [ ]:
ckd_features_df_13['Chloride'].min()

In [ ]:
ckd_features_df_13['Chloride'].max()

In [ ]:
ckd_features_df_13.loc[ckd_features_df_13['Chloride'] > 131, 'Chloride'] = np.nan
ckd_features_df_13.loc[ckd_features_df_13['Chloride'] < 80, 'Chloride'] = np.nan

In [ ]:
# Calculate the mean chloride for each age group
mean_chloride_by_age_group = ckd_features_df_13.groupby('Age_Group')['Chloride'].transform('mean')
# Impute the missing chloride values with the mean of the corresponding age group
ckd_features_df_13['Chloride'] = ckd_features_df_13['Chloride'].fillna(mean_chloride_by_age_group)

In [ ]:
ckd_features_df_13['Chloride'].hist()

## Alkaline phosphate

In [ ]:
ckd_features_df_13.rename(columns={'Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma': 'ALP'}, inplace=True)

In [ ]:
ckd_features_df_13['ALP'].hist()

In [ ]:
ckd_features_df_13['ALP'].min()

In [ ]:
ckd_features_df_13['ALP'].max()

In [ ]:
ckd_features_df_13.loc[ckd_features_df_13['ALP'] > 190, 'ALP'] = np.nan
ckd_features_df_13.loc[ckd_features_df_13['ALP'] < 1, 'ALP'] = np.nan

In [ ]:
# Calculate the mean ALP for each age group
mean_alp_by_age_group = ckd_features_df_13.groupby('Age_Group')['ALP'].transform('mean')
# Impute the missing ALP values with the mean of the corresponding age group
ckd_features_df_13['ALP'] = ckd_features_df_13['ALP'].fillna(mean_alp_by_age_group)

In [ ]:
ckd_features_df_13['ALP'].hist()

## Proteinuria

In [ ]:
# This query represents dataset "ML Features- CKD" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
dataset_76108170_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_standard_concept.concept_name as standard_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (75650)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id"""

dataset_76108170_condition_df = pd.read_gbq(
    dataset_76108170_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_76108170_condition_df.head(5)

In [ ]:
proteinuria_df = (
    dataset_76108170_condition_df
    .assign(proteinuria=dataset_76108170_condition_df['standard_concept_name'].notna().astype(int))
    .groupby('person_id', as_index=False)['proteinuria']
    .max()
)


In [ ]:
ckd_features_df_14 = (
   ckd_features_df_13.merge(proteinuria_df, on="person_id", how="left")
)

In [ ]:
ckd_features_df_14['proteinuria'].isna().sum()


In [ ]:
ckd_features_df_14['proteinuria'] = ckd_features_df_14['proteinuria'].fillna(0)


In [ ]:
ckd_features_df_14['proteinuria'].value_counts()

## Coronay Artery Disease

In [ ]:
# This query represents dataset "ML Features- CKD" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
dataset_69198462_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_standard_concept.concept_name as standard_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (317576)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 124 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        DATE_DIFF(CURRENT_DATE, dob, YEAR) - IF(EXTRACT(MONTH FROM dob)*100 + EXTRACT(DAY FROM dob) > EXTRACT(MONTH FROM CURRENT_DATE)*100 + EXTRACT(DAY FROM CURRENT_DATE), 1, 0) BETWEEN 18 AND 100 
                        AND NOT EXISTS (      SELECT
                            'x'      
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.death` d      
                        WHERE
                            d.person_id = p.person_id ) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3038553) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1585952, 1585940, 1585723, 1585375, 1585723, 1585729, 1585386, 1585952, 1585375, 1585952, 1585375, 1585375, 1585892, 1585892, 1585375, 1585940, 1585952, 1585855, 1585729, 1585375, 1740639, 1585940, 1585952, 1585375, 1585375, 1585729, 43528895, 1585723, 1585723, 1585892, 1585940, 1585892, 1585386, 1585386, 40192389, 1585729, 1585892, 1585375, 1585723, 1585952, 1585940, 1586134, 1585940, 1585386, 1585710, 1585729, 1585892, 1585892, 1585940, 1585952, 1585940, 1585940, 1585729, 1585892, 1585952, 1585940, 1585892, 1585723, 1585723, 1585729, 1585729, 1585386, 1585952, 1585375, 1585952, 1585940, 1585952, 1585386, 1585375, 1585375)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3027018) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (903118, 903115) 
                            AND is_standard = 0 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3013682) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3004501) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3019550) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN (3023103) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (443392)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (903079, 0, 903096, 1586148) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        sex_at_birth_concept_id IN (903096, 1177221, 0, 1585849, 46273637) ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        race_concept_id IN (2100000001, 903096, 45882607, 1177221) ) )
            )) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id"""

dataset_69198462_condition_df = pd.read_gbq(
    dataset_69198462_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_69198462_condition_df.head(5)

In [ ]:
cad_df = (
    dataset_69198462_condition_df
    .assign(cad=dataset_69198462_condition_df['standard_concept_name'].notna().astype(int))
    .groupby('person_id', as_index=False)['cad']
    .max()
)

In [ ]:
ckd_features_df_15 = (
   ckd_features_df_14.merge(cad_df, on="person_id", how="left")
)


In [ ]:
ckd_features_df_15['cad'].isna().sum()

In [ ]:
ckd_features_df_15['cad']= ckd_features_df_15['cad'].fillna(0)

In [ ]:
ckd_features_df_15['cad'].value_counts()

# Formatting file for GWAS 

In [ ]:
patient_df = ckd_features_df_12[['person_id']]

In [ ]:
patient_df.head(10)

In [ ]:
bucket = os.getenv("WORKSPACE_BUCKET")
bucket

Save the patient ids to a tsv file so we can analyze it in the PRS notebook. After that is complete, we will reload the dataframe with the PRS scores and merge it with the rest of the features.

In [ ]:
patient_df.to_csv('genomics_ckd_phenotypes.tsv', index=False, sep='\t')

# save phenotypes to the bucket
!gsutil cp 'genomics_ckd_phenotypes.tsv' {bucket}/data/

## Loading PRS score csv back into the environment

In [ ]:
prs_file_path = 'PRS_scores.csv'

my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{prs_file_path}' .")

print(f'[INFO] {prs_file_path} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
prs_df = pd.read_csv(prs_file_path)
prs_df.head()

In [ ]:
prs_df.rename(columns={'s': 'person_id'}, inplace=True)

In [ ]:
ckd_features_df_16 = (
   ckd_features_df_15.merge(prs_df, on="person_id", how="left")
)

In [ ]:
ckd_features_df_16.columns

## Correlation Plot

This correlation plot will inform of us of any multicollinearity 

In [ ]:
import matplotlib.pyplot as plt

corr_plot_vars = ['outcome', 'Age', 'BMI', 'dbp', 'sbp', 'Albumin', 'Creatinine', 'Glucose', 'Potassium', 'Sodium', 'Urea', 'Erythrocytes','Hemoglobin', 'Leukocytes', 'ALP', 'Calcium', 'Chloride', 'PRS']

corr_plot = ckd_features_df_16[corr_plot_vars]

plt.figure(figsize=(14,12))
sns.heatmap(corr_plot.corr(), annot=True, fmt=".2f", linewidths=0.5, cmap="coolwarm")
plt.title("Correlation Heatmap", fontsize=18)
plt.show()
plot_filename = 'correlation_heatmap.png'
plt.savefig(plot_filename, bbox_inches='tight')

## Categorical Visualization

In [ ]:
def plot_categorical_variables(df, categorical_vars, n_cols=3, row_height=10, figsize=(15, 20)):
    
    n_rows = (len(categorical_vars) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()
    for i, var in enumerate(categorical_vars):
        sns.countplot(data=df, x=var, ax=axes[i], palette='pastel')
        axes[i].set_title(f'Distribution of {var}', fontsize=14)
        axes[i].tick_params(axis='x', rotation=45)
    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
categorical_vars = ['race', 'ethnicity', 'sex', 'smoking_status','alcohol_score', 'mental_health', 'physical_health', 'afford_provider', 'diabetes', 'hypertension', 'anemia', 'proteinuria', 'cad']

plot_categorical_variables(ckd_features_df_16, categorical_vars)

## Check for normality and transform nonnormal variables

I will transform any continuous variables with a skew > 0.5.
I am using the box cox transformation method

In [ ]:
from scipy.stats import skew, boxcox

def normalize_skewed_variables(df, continuous_vars, skew_thresh=0.5):
    transformations = {}

    for col in continuous_vars:
        data = df[col]
        skewness = skew(data)

        if abs(skewness) > skew_thresh:
            try:
                # Apply Box-Cox transformation and replace the original column
                transformed_data, _ = boxcox(data)
                df[col] = transformed_data  # Directly replace the original column
                transformations[col] = 'Box-Cox transformation applied'
            except:
                transformations[col] = f'Box-Cox failed for {col} - no transformation'
        else:
            transformations[col] = f'{col} - no significant skewness, no transformation'

    return df, transformations

In [ ]:
continuous_vars = ['Age', 'BMI', 'dbp', 'sbp', 'Albumin', 'Creatinine', 
                   'Glucose', 'Potassium', 'Sodium', 'Urea', 'Erythrocytes', 
                   'Hemoglobin', 'Leukocytes', 'ALP', 'Calcium', 'Chloride','PRS']


# Apply normalization
ckd_features_df_16, transformations = normalize_skewed_variables(ckd_features_df_16, continuous_vars)

print(transformations)

In [ ]:
def plot_histograms(df, continuous_vars, bins=20):
    plt.figure(figsize=(15, 12))  
    for i, col in enumerate(continuous_vars, 1):
        plt.subplot(4, 5, i)  
        sns.histplot(df[col], bins=bins, kde=True)  
        plt.title(f'{col} Histogram')  # Title for each subplot
        plt.tight_layout() 

    plt.show()
    
plot_histograms(ckd_features_df_16, continuous_vars)


## Apply standard scalar 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
ckd_features_df_16[continuous_vars] = scaler.fit_transform(ckd_features_df_16[continuous_vars])

In [ ]:
plot_histograms(ckd_features_df_16, continuous_vars)


## One hot encoding categorical variables

In [ ]:
def one_hot_encode(df, categorical_vars, drop_first=True):
    df_encoded = pd.get_dummies(df, columns=categorical_vars, drop_first=drop_first)
    return df_encoded

In [ ]:
categorical_vars = ['race']

In [ ]:
ckd_features_df_17 = one_hot_encode(ckd_features_df_16, categorical_vars)

In [ ]:
ckd_features_df_17.head()

## Apply final cleaning

- No longer need the Age Group bins as those were used to impute missing data
- Encode ethnicity to 0 and 1
- Drop any other unnecessary columns

In [ ]:
ethnicity_mapping = {
    'Not Hispanic or Latino': 0,
    'Hispanic or Latino': 1
}

ckd_features_df_17['ethnicity'] = ckd_features_df_17['ethnicity'].map(ethnicity_mapping)

In [ ]:
ckd_features_df_17['ethnicity'].value_counts()

In [ ]:
ckd_features_df_17 = ckd_features_df_17.drop(columns = ['condition_type_concept_id', 'Age_Group'])

In [ ]:
ckd_features_df_17 = ckd_features_df_17.drop(columns = ['low_gfr'])

## Save final ML features df to the workspace 

That way we can create the model in a separate notebook

In [ ]:
ckd_features_df_17.to_csv('features.csv', index=False)

# save PRS scores to the bucket
!gsutil cp 'features.csv' {bucket}/data/